In [1]:
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
day = datetime.datetime.strftime(datetime.datetime.now(), "%Y_%m_%d")
day

'2022_08_19'

In [3]:

! rm -rf "eqList/eqList"$day".xls"
! cp "/Users/ivan/Downloads/eqList"$day".xls" "eqList/eqList"$day".xls"
! ls -l "eqList/"

aeta = f"eqList/eqList{day}.xls"
aeta

total 45216
-rw-r--r--@ 1 ivan  staff   565464  7 30 18:56 eqList2022_07_30.xlsx
-rw-r--r--@ 1 ivan  staff   566439  8  9 01:05 eqList2022_08_09.xlsx
-rw-r--r--@ 1 ivan  staff  3567272  8 11 11:27 eqList2022_08_11.xls
-rw-r--r--@ 1 ivan  staff   566428  8 11 00:45 eqList2022_08_11.xlsx
-rw-r--r--@ 1 ivan  staff  3568288  8 12 16:56 eqList2022_08_12.xls
-rw-r--r--@ 1 ivan  staff  3569648  8 13 15:09 eqList2022_08_13.xls
-rw-r--r--@ 1 ivan  staff  3573668  8 15 16:06 eqList2022_08_15.xls
-rw-r--r--@ 1 ivan  staff  3575011  8 16 18:58 eqList2022_08_16.xls
-rw-r--r--@ 1 ivan  staff  3576341  8 19 17:35 eqList2022_08_19.xls


'eqList/eqList2022_08_19.xls'

In [4]:
N = 7
T = 10
# 经度
lonL, lonH = 98, 107
# 纬度
latL, latH = 22, 34 


In [5]:

import xml
DOMTree = xml.dom.minidom.parse(aeta).documentElement
Row = DOMTree.getElementsByTagName("Row")

cols, data = [], []
for nRow, iRow in enumerate(Row):
    if nRow == 0:
        for jRow in iRow.getElementsByTagName("Cell"):
            cols.append(jRow.childNodes[0].childNodes[0].data)
    else:
        idata = []
        for jRow in iRow.getElementsByTagName("Cell"):
            idata.append(jRow.childNodes[0].childNodes[0].data)
        data.append([float(dj) if nj in [1,2,3,4] else str(dj) for nj ,dj in enumerate(idata)])

data = pd.DataFrame(data, columns=cols)
print(data.dtypes)
data.head(T)


发震时刻       object
震级(M)     float64
纬度(°)     float64
经度(°)     float64
深度(千米)    float64
参考位置       object
dtype: object


,发震时刻,震级(M),纬度(°),经度(°),深度(千米),参考位置
0,2022-08-18 15:46:27,5.4,0.30,124.40,150.0,印尼北苏拉威西省附近海域
1,2022-08-18 10:10:21,5.3,-34.40,-70.60,100.0,智利
2,2022-08-17 20:56:32,5.6,-7.15,125.55,500.0,班达海
3,2022-08-17 12:24:45,5.3,33.35,142.10,10.0,日本本州东岸远海
4,2022-08-16 10:22:55,4.3,33.14,90.27,10.0,西藏那曲市安多县
5,2022-08-16 10:16:53,3.0,33.04,90.38,10.0,西藏那曲市安多县
6,2022-08-15 21:05:09,4.1,33.12,90.38,10.0,西藏那曲市安多县
7,2022-08-15 16:30:37,4.7,38.43,99.26,8.0,青海海北州祁连县
8,2022-08-15 05:10:36,4.6,37.85,93.72,10.0,青海海西州直辖区
9,2022-08-15 05:04:48,6.4,-22.00,170.95,100.0,洛亚蒂群岛


In [6]:
def regeo(location):
    try:
        import json
        import requests

        host = 'https://regeo.market.alicloudapi.com'
        path = '/v3/geocode/regeo'
        appcode = '830fa522a512421eaa202bb80afe8921'
        querys = f'location={location}' # 经度在前，纬度在后
        with requests.get(
            f"{host}{path}?{querys}", 
            headers={'Authorization': f'APPCODE {appcode}'}
        ) as r:
            j = json.loads(r.content.decode())["regeocode"]["formatted_address"]
            return j
    except:
        return "NaN"

In [7]:

Kdata = []
for STATION in tqdm(data.head(T)["参考位置"]):
    # idata
    idata = data[data["参考位置"] == STATION][["参考位置", "发震时刻"]]
    idata.rename({
        "参考位置": "初次参考位置", 
        "发震时刻": "初次发震时刻",
    }, axis=1, inplace=True)
    idata["K"] = 1
    # print(idata.shape)

    # jdata
    jdata = data[
        (data["参考位置"] != STATION) & 
        (data["经度(°)"] >= lonL) & (data["经度(°)"] <= lonH) &
        (data["纬度(°)"] >= latL) & (data["纬度(°)"] <= latH) &
        # (data["震级(M)"] >= 3.5) &
        True
    ][["参考位置", "震级(M)", "纬度(°)", "经度(°)", "发震时刻"]]
    jdata["K"] = 1
    # print(jdata.shape)

    # k0data
    k0data = pd.merge(
        idata,
        jdata,
        on="K"
    )
    # print(k0data.shape)

    # k1data
    # 筛选近N天内
    k0data["gap-发震时刻"] = (
        pd.to_datetime(k0data["发震时刻"]) - 
        pd.to_datetime(k0data["初次发震时刻"])
    ).dt.days

    k1data = (k0data["gap-发震时刻"] >= 1) & (k0data["gap-发震时刻"] <= N)
    # print(pd.value_counts(k1data))

    k1data = k0data[k1data]
    # print(k1data.shape)

    # k2data
    k2data = k1data.groupby(
        ["初次参考位置", "初次发震时刻", "K"], 
        as_index=False
    ).agg({
        "震级(M)": "mean", # 特殊/余震
        "纬度(°)": "mean",
        "经度(°)": "mean",
    }).groupby(
        ["K"], 
        as_index=False
    ).agg(
        {
        "K": "sum",
        "震级(M)": "mean", 
        "纬度(°)": "mean",
        "经度(°)": "mean",
    })

    k2data["参考位置"] = STATION
    k2data["K%"] = k2data["K"]/idata.shape[0]
    k2data["震级C"] = k2data["震级(M)"].apply(lambda x: "" if x >= 3.5 else "❌")
    k2data["经度C"] = k2data["经度(°)"].apply(lambda x: "" if x >= lonL and x <= lonH else "❌")
    k2data["纬度C"] = k2data["纬度(°)"].apply(lambda x: "" if x >= latL and x <= latH else "❌")
    k2data["经纬度解析"] = [regeo(f"{_1},{_2}") for _1, _2 in zip(k2data["经度(°)"], k2data["纬度(°)"])]
    k2data["经度(°)"] = k2data["经度(°)"].apply(lambda x: f"{x:.6f}")
    k2data["纬度(°)"] = k2data["纬度(°)"].apply(lambda x: f"{x:.6f}")

    k2data = k2data[
        ['参考位置', '震级(M)', '震级C', '纬度(°)', '纬度C', '经度(°)', '经度C', 'K', 'K%','经纬度解析']
    ]
    # print(k2data)
    Kdata.append(k2data)

    # 
    del idata, jdata, k0data, k1data, k2data


100%|██████████| 10/10 [00:13<00:00,  1.37s/it]


In [8]:
endata = pd.merge(
    data.head(T).rename(
        {icol: f"#{icol}#" for icol in data.columns if icol != "参考位置"}, 
        axis=1
    ), 
    pd.concat(Kdata),
    on="参考位置"
).sort_values("#发震时刻#", ascending=False)

endata

,#发震时刻#,#震级(M)#,#纬度(°)#,#经度(°)#,#深度(千米)#,参考位置,震级(M),震级C,纬度(°),纬度C,经度(°),经度C,K,K%,经纬度解析
0,2022-08-18 15:46:27,5.4,0.30,124.40,150.0,印尼北苏拉威西省附近海域,3.651795,,29.607564,,102.082769,,3,0.600000,四川省甘孜藏族自治州泸定县磨西镇海螺沟冰川森林公园
1,2022-08-18 10:10:21,5.3,-34.40,-70.60,100.0,智利,3.281373,❌,29.369804,,103.348431,,17,0.459459,四川省乐山市峨眉山市龙门乡003乡道
2,2022-08-17 20:56:32,5.6,-7.15,125.55,500.0,班达海,3.387615,❌,28.126688,,102.282701,,23,0.766667,四川省凉山彝族自治州喜德县红莫镇格车组
3,2022-08-17 12:24:45,5.3,33.35,142.10,10.0,日本本州东岸远海,3.665152,,28.068788,,103.298788,,11,0.523810,四川省凉山彝族自治州雷波县莫红乡觉吉村
4,2022-08-16 10:22:55,4.3,33.14,90.27,10.0,西藏那曲市安多县,3.318333,❌,27.316729,,101.975781,,16,0.666667,四川省凉山彝族自治州德昌县热河镇团石坝
5,2022-08-16 10:22:55,4.3,33.14,90.27,10.0,西藏那曲市安多县,3.318333,❌,27.316729,,101.975781,,16,0.666667,四川省凉山彝族自治州德昌县热河镇团石坝
6,2022-08-16 10:22:55,4.3,33.14,90.27,10.0,西藏那曲市安多县,3.318333,❌,27.316729,,101.975781,,16,0.666667,四川省凉山彝族自治州德昌县热河镇团石坝
7,2022-08-16 10:16:53,3.0,33.04,90.38,10.0,西藏那曲市安多县,3.318333,❌,27.316729,,101.975781,,16,0.666667,四川省凉山彝族自治州德昌县热河镇团石坝
8,2022-08-16 10:16:53,3.0,33.04,90.38,10.0,西藏那曲市安多县,3.318333,❌,27.316729,,101.975781,,16,0.666667,四川省凉山彝族自治州德昌县热河镇团石坝
9,2022-08-16 10:16:53,3.0,33.04,90.38,10.0,西藏那曲市安多县,3.318333,❌,27.316729,,101.975781,,16,0.666667,四川省凉山彝族自治州德昌县热河镇团石坝


In [9]:
# 
day1 = datetime.datetime.strftime(datetime.datetime.now() + datetime.timedelta(days=1), "%Y-%m-%d")
dayn = datetime.datetime.strftime(datetime.datetime.now() + datetime.timedelta(days=N), "%Y-%m-%d")
day7 = datetime.datetime.strftime(datetime.datetime.now() + datetime.timedelta(days=7), "%Y-%m-%d")
print(f">>> {day1} To {day7} / {dayn}")

message = f"""
# 有震预测 earthquake prediction
check_my_prediction(myToken, '{day1}', '{dayn}', 1, latitude=$_1, longitude=$_2, magnitude=$_3)
"""
endata = endata[endata["震级C"] != "❌"]
for _1, _2, _3 in zip(endata["纬度(°)"], endata["经度(°)"], endata["震级(M)"]):
    print(message
          .replace("$_1",f"{float(_1):.6f}")
          .replace("$_2",f"{float(_2):.6f}")
          .replace("$_3",f"{float(_3):.1f}")
         )

message = f"""
# 无震预测 No earthquake prediction
check_my_prediction(myToken, '{day1}', '{day7}', 0)
"""
print(message)
print()


>>> 2022-08-20 To 2022-08-26 / 2022-08-22

# 有震预测 earthquake prediction
check_my_prediction(myToken, '2022-08-20', '2022-08-22', 1, latitude=29.607564, longitude=102.082769, magnitude=3.7)


# 有震预测 earthquake prediction
check_my_prediction(myToken, '2022-08-20', '2022-08-22', 1, latitude=28.068788, longitude=103.298788, magnitude=3.7)


# 有震预测 earthquake prediction
check_my_prediction(myToken, '2022-08-20', '2022-08-22', 1, latitude=29.816143, longitude=102.714429, magnitude=3.5)


# 无震预测 No earthquake prediction
check_my_prediction(myToken, '2022-08-20', '2022-08-26', 0)




In [10]:
! cp Test002.ipynb "back/Test002-"$day".ipynb"
! ls -l back/

total 144
-rw-r--r--@ 1 ivan  staff  34490  8 16 18:58 Test002-2022_08_16.ipynb
-rw-r--r--@ 1 ivan  staff  34489  8 19 17:35 Test002-2022_08_19.ipynb
